# Exploring 72 records from 1543 (Kickback from Lex's R Code)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 15, 2021
***
`This Jupyter Notebook explores 72 records (of the 1543 OTHER records) that were returned after being ran through Lex's R Code. The process for exploring these records is as follows:`

1. Explore the 'DATASET' field, which indicates what type of Origin Location Reference appears within the 'REMARKS' field for each record (e.g., AIRPORT, NAVAID, AIRPORT DIRECTLY OVER, NAVAID DIRECTLY OVER, RUNWAY, FIX RADIAL, HELIPORT, etc) 
<br>

2. For each of the unique Origin Location References that appear in the 'DATASET' field:

    a. Run the uas_loc_parts() function of the dataframe 
    
    b. Link to the appropriate Origin Location Reference linking function (e.g., link_airportsCleaned_ident(), link_airportsCleaned_icao(), nav_link())
        i. Separate the field that were successfully linked with the Origin Location Reference Dataset(s) and those that were not. Those that were not linked successfully will be moved to the exceptions file for further review.
    c. Run the successfully linked dataframe through the uas_lat_long() function to calculate the UAS Latitude/Longitude coordinates for each respective record/remark. 

***
### INPUT: 
*Dataframe (read from a .csv file) containing 72 records, of 1543 records that did not hit on the initial 'XX NM dir XXX' regular expressions, and was not initially picked up by our Runways regular expressions.*

### OUTPUT:
<font color='red'>**EXCPETIONS FILE**</font>

*ICAO Airport Identifier: 6 records (no match when linking with Airports)*

*Directly Over Airport: 12 records (no match when linking with Airports)*

*Directly over NAVAID Systems: 1 record (no match when linking with NAVAID)*

*NAVAID System: 2 records (no match when linking with NAVAID)*

*Runway: 4 records (no specific runway is mentioned)*

<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*FAA Airport Identifier: 3 records*

*Directly Over Airport: 36 records*

*Directly over NAVAID Systems: 2 records*

*NAVAID System: 4 records*

*Fix/Raidal/Distance: 1 record*

*Heliport: 1 record*


## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

In [2]:
# Read in file containing 72 kickback records from Lex's R Code
output72 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace - All_Points_Standard_72OUTOF382 (from the 415).csv')
output72

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,156,"Aircraft observed a uas at 12,000 feet 2 NM W ...",UNKN,7/26/2019,23:45,2019-07-26T23:45Z,DEN,UAS,NaN,MKE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 NM W of C89,AIRPORT,NaN
1,670,Aircraft observed a black uas off the right si...,UNKN,7/19/2020,18:33,2020-07-19T18:33Z,DEN,UAS,NaN,ACT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5 NM S of KT15,AIRPORT,NaN
2,817,Aircraft observed a red and black uas off the ...,UNKN,9/26/2020,20:49,2020-09-26T20:49Z,DEN,UAS,NaN,AUS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 NM NE of K50R,AIRPORT,NaN
3,818,"Aircraft observed a large, white uas off the l...",UNKN,9/26/2020,22:14,2020-09-26T22:14Z,DEN,UAS,NaN,A90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 NM E of 26MA,AIRPORT,NaN
4,1023,Aircraft observed a pale uas off the left side...,UNKN,3/6/2021,23:07,2021-03-06T23:07Z,NCRCC,UAS,NaN,NCRCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM S of K2W5,AIRPORT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,922,Aircraft observed a uas at same altitude while...,UNKN,12/29/2020,18:23,2020-12-29T18:23Z,DEN,UAS,NaN,DWH,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 17R,DWH,RUNWAY,NaN
68,1376,"Aircraft observed a uas at 12 o clock, 100 fee...",UNKN,6/21/2021,13:50,2021-06-21T13:50Z,NCRCC,UAS,NaN,IAD,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 19L,4 NM final to IAD,RUNWAY,NaN
69,416,"Aircraft observed a 2 uas.s, grey and white, o...",UNKN,2/16/2020,15:27,2020-02-16T15:27Z,NCRCC,UAS,NaN,BWI,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 33L,2 NM final to BWI,RUNWAY,NaN
70,508,Aircraft observed a black uas while S bound at...,UNKN,5/16/2020,15:03,2020-05-16T15:03Z,DEN,UAS,NaN,MIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20 NM ENE of DHP,NAVAID,NaN


In [3]:
output72_df = output72[['REMARKS', 'DATASET', 'UASLOCATION', 'cityname']]
output72_df

,REMARKS,DATASET,UASLOCATION,cityname
0,"Aircraft observed a uas at 12,000 feet 2 NM W ...",AIRPORT,2 NM W of C89,NaN
1,Aircraft observed a black uas off the right si...,AIRPORT,12.5 NM S of KT15,NaN
2,Aircraft observed a red and black uas off the ...,AIRPORT,7 NM NE of K50R,NaN
3,"Aircraft observed a large, white uas off the l...",AIRPORT,1 NM E of 26MA,NaN
4,Aircraft observed a pale uas off the left side...,AIRPORT,5 NM S of K2W5,NaN
...,...,...,...,...
67,Aircraft observed a uas at same altitude while...,RUNWAY,DWH,NaN
68,"Aircraft observed a uas at 12 o clock, 100 fee...",RUNWAY,4 NM final to IAD,NaN
69,"Aircraft observed a 2 uas.s, grey and white, o...",RUNWAY,2 NM final to BWI,NaN
70,Aircraft observed a black uas while S bound at...,NAVAID,20 NM ENE of DHP,NaN


In [4]:
# Determine Unique Origin Location References within the 72 record kickback
output72_df['DATASET'].unique()

array(['AIRPORT', 'AIRPORT DIRECTLY OVER', 'FIX RADIAL', 'HELIPORT',
       'NAVAID', 'NAVAID DIRECTLY OVER', 'RUNWAY'], dtype=object)

# AIRPORTS

In [5]:
output72_airport = output72_df[output72_df['DATASET'] == 'AIRPORT'].drop(columns = ['cityname']).reset_index(drop=True)
output72_airport

,REMARKS,DATASET,UASLOCATION
0,"Aircraft observed a uas at 12,000 feet 2 NM W ...",AIRPORT,2 NM W of C89
1,Aircraft observed a black uas off the right si...,AIRPORT,12.5 NM S of KT15
2,Aircraft observed a red and black uas off the ...,AIRPORT,7 NM NE of K50R
3,"Aircraft observed a large, white uas off the l...",AIRPORT,1 NM E of 26MA
4,Aircraft observed a pale uas off the left side...,AIRPORT,5 NM S of K2W5
5,Aircraft observed a uas or balloon off the lef...,AIRPORT,7 NM W of KB16
6,"Aircraft observed a uas 3,000 feet below while...",AIRPORT,9 NM N of K1A9
7,Aircraft observed a quadcopter uas 100 feet be...,AIRPORT,1 NM E of 4M8
8,Aircraft observed a blue uas while N bound at ...,AIRPORT,2 NM N of K3GM


In [6]:
def uas_loc_parts(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    ident_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get IDENT from UASLOCATION
        ident = uasLoc_col[i].split()[-1].replace('of', '')
        ident_list.append(ident)
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        bearing = re.sub('(NM|of|OF)', ' ', uasLoc_col[i]).split()[-2]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['IDENT'] = ident_list
    
    
    return dataframe

### FAA Airport Identifier

`These 3 records were successfully parsed for UAS Lat/Long calculations.`

In [7]:
# Call the uas_loc_parts() function on the records with an Airport Origin Location Reference
air9 = uas_loc_parts(output72_airport)
air9

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,"Aircraft observed a uas at 12,000 feet 2 NM W ...",AIRPORT,2 NM W of C89,2,3.704,W,270,C89
1,Aircraft observed a black uas off the right si...,AIRPORT,12.5 NM S of KT15,12.5,23.150,S,180,KT15
2,Aircraft observed a red and black uas off the ...,AIRPORT,7 NM NE of K50R,7,12.964,NE,45,K50R
3,"Aircraft observed a large, white uas off the l...",AIRPORT,1 NM E of 26MA,1,1.852,E,90,26MA
4,Aircraft observed a pale uas off the left side...,AIRPORT,5 NM S of K2W5,5,9.260,S,180,K2W5
5,Aircraft observed a uas or balloon off the lef...,AIRPORT,7 NM W of KB16,7,12.964,W,270,KB16
6,"Aircraft observed a uas 3,000 feet below while...",AIRPORT,9 NM N of K1A9,9,16.668,N,0,K1A9
7,Aircraft observed a quadcopter uas 100 feet be...,AIRPORT,1 NM E of 4M8,1,1.852,E,90,4M8
8,Aircraft observed a blue uas while N bound at ...,AIRPORT,2 NM N of K3GM,2,3.704,N,0,K3GM


In [8]:
def link_airportsCleaned_ident(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='inner')

In [9]:
# Link on airports dataset on the IDENT field
faa_air3 = link_airportsCleaned_ident(air9)
faa_air3

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,"Aircraft observed a uas at 12,000 feet 2 NM W ...",AIRPORT,2 NM W of C89,2,3.704,W,270,C89,51E11B6D-3CC5-48FD-938B-3C3E3CDFB30C,Sylvania,42.703250,-87.958972,NaN
1,"Aircraft observed a large, white uas off the l...",AIRPORT,1 NM E of 26MA,1,1.852,E,90,26MA,D3CA8340-1C80-4535-AFCB-20713184AC9B,Pepperell,42.696194,-71.550056,NaN
2,Aircraft observed a quadcopter uas 100 feet be...,AIRPORT,1 NM E of 4M8,1,1.852,E,90,4M8,F78DE99F-720B-4627-B360-DF16F10BCCD5,Clarendon Muni,34.646589,-91.394717,NaN


In [10]:
def uas_lat_long(dataframe):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param dataframe : The dataframe that includes the fields 'LATITUDE', 'LONGITUDE', 'Bearing_Degrees', 
                        'Distance_Kilometers' to have UAS Lat/Long values calculated for
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    uas_lat = []
    uas_long = []
    
    for i in range(len(dataframe)):
        lat_airport = pd.to_numeric(dataframe['LATITUDE'][i])
        long_airport = pd.to_numeric(dataframe['LONGITUDE'][i])
        b = pd.to_numeric(dataframe['Bearing_Degrees'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2 = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
        
    
    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    #dataframe.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return dataframe

In [11]:
# Calculate UAS Lat/Long coordinates
faa_air3_uas = uas_lat_long(faa_air3)
faa_air3_uas

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,"Aircraft observed a uas at 12,000 feet 2 NM W ...",AIRPORT,2 NM W of C89,2,3.704,W,270,C89,51E11B6D-3CC5-48FD-938B-3C3E3CDFB30C,Sylvania,42.703250,-87.958972,NaN,42.703241,-88.004180
1,"Aircraft observed a large, white uas off the l...",AIRPORT,1 NM E of 26MA,1,1.852,E,90,26MA,D3CA8340-1C80-4535-AFCB-20713184AC9B,Pepperell,42.696194,-71.550056,NaN,42.696192,-71.527454
2,Aircraft observed a quadcopter uas 100 feet be...,AIRPORT,1 NM E of 4M8,1,1.852,E,90,4M8,F78DE99F-720B-4627-B360-DF16F10BCCD5,Clarendon Muni,34.646589,-91.394717,NaN,34.646587,-91.374516


In [12]:
# Re merge with original dataset to obtain all rows
faa_air3_uasdf = pd.merge(faa_air3_uas, output72, on='REMARKS', how='left')

In [13]:
# Export to .csv 
faa_air3_uasdf.to_csv('air03OF72_uasLatLong.csv', index = False)

### ICAO Airport Identifier

`It should be noted that these 6 records will be moved to the exceptions file, as their ICAO_ID does not match with any of those present in the airports_cleaned.csv dataset`

In [14]:
air9 = uas_loc_parts(output72_airport)
#air9

In [15]:
# Only keep records for which the Origin Location References an Airport by its ICAO Identifier
ident = air9['IDENT']
icao = []

for i in range(len(air9)):
    if ident[i][0] == 'K':
        icao.append(ident[i])
    else:
        icao.append(np.nan)

air9['ICAO_ID'] = icao
air9 = air9.dropna().reset_index(drop=True)
air9

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,ICAO_ID
0,Aircraft observed a black uas off the right si...,AIRPORT,12.5 NM S of KT15,12.5,23.150,S,180,KT15,KT15
1,Aircraft observed a red and black uas off the ...,AIRPORT,7 NM NE of K50R,7,12.964,NE,45,K50R,K50R
2,Aircraft observed a pale uas off the left side...,AIRPORT,5 NM S of K2W5,5,9.260,S,180,K2W5,K2W5
3,Aircraft observed a uas or balloon off the lef...,AIRPORT,7 NM W of KB16,7,12.964,W,270,KB16,KB16
4,"Aircraft observed a uas 3,000 feet below while...",AIRPORT,9 NM N of K1A9,9,16.668,N,0,K1A9,K1A9
5,Aircraft observed a blue uas while N bound at ...,AIRPORT,2 NM N of K3GM,2,3.704,N,0,K3GM,K3GM


In [16]:
def link_airportsCleaned_icao(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='ICAO_ID', how='left')

In [17]:
# Link to airports dataset
icao_air6 = link_airportsCleaned_icao(air9)
icao_air6

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT_x,ICAO_ID,GLOBAL_ID,IDENT_y,NAME,LATITUDE,LONGITUDE
0,Aircraft observed a black uas off the right si...,AIRPORT,12.5 NM S of KT15,12.5,23.150,S,180,KT15,KT15,NaN,NaN,NaN,NaN,NaN
1,Aircraft observed a red and black uas off the ...,AIRPORT,7 NM NE of K50R,7,12.964,NE,45,K50R,K50R,NaN,NaN,NaN,NaN,NaN
2,Aircraft observed a pale uas off the left side...,AIRPORT,5 NM S of K2W5,5,9.260,S,180,K2W5,K2W5,NaN,NaN,NaN,NaN,NaN
3,Aircraft observed a uas or balloon off the lef...,AIRPORT,7 NM W of KB16,7,12.964,W,270,KB16,KB16,NaN,NaN,NaN,NaN,NaN
4,"Aircraft observed a uas 3,000 feet below while...",AIRPORT,9 NM N of K1A9,9,16.668,N,0,K1A9,K1A9,NaN,NaN,NaN,NaN,NaN
5,Aircraft observed a blue uas while N bound at ...,AIRPORT,2 NM N of K3GM,2,3.704,N,0,K3GM,K3GM,NaN,NaN,NaN,NaN,NaN


In [18]:
# Merge with original dataframe to obtain all rows
icao_air6df = pd.merge(icao_air6, output72, on='REMARKS', how='left')
icao_air6df

,REMARKS,DATASET_x,UASLOCATION_x,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT_x,ICAO_ID,GLOBAL_ID,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a black uas off the right si...,AIRPORT,12.5 NM S of KT15,12.5,23.150,S,180,KT15,KT15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5 NM S of KT15,AIRPORT,NaN
1,Aircraft observed a red and black uas off the ...,AIRPORT,7 NM NE of K50R,7,12.964,NE,45,K50R,K50R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 NM NE of K50R,AIRPORT,NaN
2,Aircraft observed a pale uas off the left side...,AIRPORT,5 NM S of K2W5,5,9.260,S,180,K2W5,K2W5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM S of K2W5,AIRPORT,NaN
3,Aircraft observed a uas or balloon off the lef...,AIRPORT,7 NM W of KB16,7,12.964,W,270,KB16,KB16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 NM W of KB16,AIRPORT,NaN
4,"Aircraft observed a uas 3,000 feet below while...",AIRPORT,9 NM N of K1A9,9,16.668,N,0,K1A9,K1A9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9 NM N of K1A9,AIRPORT,NaN
5,Aircraft observed a blue uas while N bound at ...,AIRPORT,2 NM N of K3GM,2,3.704,N,0,K3GM,K3GM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 NM N of K3GM,AIRPORT,NaN


In [19]:
# Export to .csv 
icao_air6df.to_csv('icao06OF72_EXCEPTIONS.csv', index = False)

# AIRPORT DIRECTLY OVER

`These records will use the Airport Lat/Long as the UAS Lat/Long, as the UAS was described as being 'directly over' the airport in question`

In [20]:
output72_airportDO = output72_df[output72_df['DATASET'] == 'AIRPORT DIRECTLY OVER'].drop(columns = ['cityname']).reset_index(drop=True)
output72_airportDO

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a black uas 4-5 feet in diam...,AIRPORT DIRECTLY OVER,DFW
1,"Aircraft observed a uas while WNW bound at 4,5...",AIRPORT DIRECTLY OVER,GSH
2,"Aircraft reported a black, quad copter uas wit...",AIRPORT DIRECTLY OVER,CRG
3,Aircraft reported a uas hovering over JFK whil...,AIRPORT DIRECTLY OVER,JFK
4,Aircraft reported a uas or balloon while ESE a...,AIRPORT DIRECTLY OVER,RHV
5,Aircraft reported a uas at the same altitude w...,AIRPORT DIRECTLY OVER,ABQ
6,"Aircraft observed a black quad-copter uas 1,00...",AIRPORT DIRECTLY OVER,SFO
7,Aircraft observed a small uas off the front si...,AIRPORT DIRECTLY OVER,SCX
8,Aircraft observed a blue uas off the left side...,AIRPORT DIRECTLY OVER,VRB
9,Aircraft observed a uas at the 12 o'clock posi...,AIRPORT DIRECTLY OVER,ATL


In [21]:
output72_airportDO['IDENT'] = output72_airportDO['UASLOCATION']
output72_airportDO

,REMARKS,DATASET,UASLOCATION,IDENT
0,Aircraft observed a black uas 4-5 feet in diam...,AIRPORT DIRECTLY OVER,DFW,DFW
1,"Aircraft observed a uas while WNW bound at 4,5...",AIRPORT DIRECTLY OVER,GSH,GSH
2,"Aircraft reported a black, quad copter uas wit...",AIRPORT DIRECTLY OVER,CRG,CRG
3,Aircraft reported a uas hovering over JFK whil...,AIRPORT DIRECTLY OVER,JFK,JFK
4,Aircraft reported a uas or balloon while ESE a...,AIRPORT DIRECTLY OVER,RHV,RHV
5,Aircraft reported a uas at the same altitude w...,AIRPORT DIRECTLY OVER,ABQ,ABQ
6,"Aircraft observed a black quad-copter uas 1,00...",AIRPORT DIRECTLY OVER,SFO,SFO
7,Aircraft observed a small uas off the front si...,AIRPORT DIRECTLY OVER,SCX,SCX
8,Aircraft observed a blue uas off the left side...,AIRPORT DIRECTLY OVER,VRB,VRB
9,Aircraft observed a uas at the 12 o'clock posi...,AIRPORT DIRECTLY OVER,ATL,ATL


In [22]:
def link_airportsCleaned_ident(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='left')

In [23]:
air36_link = link_airportsCleaned_ident(output72_airportDO)
air36_link

,REMARKS,DATASET,UASLOCATION,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed a black uas 4-5 feet in diam...,AIRPORT DIRECTLY OVER,DFW,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
1,"Aircraft observed a uas while WNW bound at 4,5...",AIRPORT DIRECTLY OVER,GSH,GSH,61E7F3D5-EF71-4A07-A204-30717CE5F692,Goshen Muni,41.526133,-85.794270,KGSH
2,"Aircraft reported a black, quad copter uas wit...",AIRPORT DIRECTLY OVER,CRG,CRG,0AAAC347-E1B1-4C82-8973-27920D299060,Jacksonville Exec At Craig,30.336333,-81.514444,KCRG
3,Aircraft reported a uas hovering over JFK whil...,AIRPORT DIRECTLY OVER,JFK,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK
4,Aircraft reported a uas or balloon while ESE a...,AIRPORT DIRECTLY OVER,RHV,RHV,B8849DA0-DA5F-4C6B-96ED-12E9C31F970C,Reid-Hillview of Santa Clara County,37.332861,-121.819806,KRHV
5,Aircraft reported a uas at the same altitude w...,AIRPORT DIRECTLY OVER,ABQ,ABQ,FC50BF53-167B-49DE-AF44-7D15FFC9E959,Albuquerque Intl Sunport,35.038932,-106.608262,KABQ
6,"Aircraft observed a black quad-copter uas 1,00...",AIRPORT DIRECTLY OVER,SFO,SFO,AF8BC34D-2D11-4CDB-9E87-33A0D430C055,San Francisco Intl,37.618806,-122.375417,KSFO
7,Aircraft observed a small uas off the front si...,AIRPORT DIRECTLY OVER,SCX,SCX,97B4CC8C-CC38-4D79-86F3-639102E2ACC2,Scott Muni,36.455694,-84.585750,KSCX
8,Aircraft observed a blue uas off the left side...,AIRPORT DIRECTLY OVER,VRB,VRB,F83408DF-2239-4254-9294-49C814E218A7,Vero Beach Rgnl,27.655556,-80.417944,KVRB
9,Aircraft observed a uas at the 12 o'clock posi...,AIRPORT DIRECTLY OVER,ATL,ATL,90091CE7-1372-4182-BA14-366919A3F134,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL


In [24]:
air12_exp = air36_link[air36_link['NAME'].isnull()].reset_index(drop=True)
air12_exp

,REMARKS,DATASET,UASLOCATION,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed a uas off the left side whil...,AIRPORT DIRECTLY OVER,VIC,VIC,NaN,NaN,NaN,NaN,NaN
1,Aircraft observed a white uas while W bound at...,AIRPORT DIRECTLY OVER,CRI,CRI,NaN,NaN,NaN,NaN,NaN
2,Aircraft observed a black/red uas off the righ...,AIRPORT DIRECTLY OVER,JAK,JAK,NaN,NaN,NaN,NaN,NaN
3,Aircraft observed a uas off the right side whi...,AIRPORT DIRECTLY OVER,PXR,PXR,NaN,NaN,NaN,NaN,NaN
4,"Aircraft observed a uas while S bound at 2,500...",AIRPORT DIRECTLY OVER,VAL,VAL,NaN,NaN,NaN,NaN,NaN
5,Aircraft observed a uas off the right side whi...,AIRPORT DIRECTLY OVER,CGT,CGT,NaN,NaN,NaN,NaN,NaN
6,Aircraft observed a black uas from the front s...,AIRPORT DIRECTLY OVER,CQY,CQY,NaN,NaN,NaN,NaN,NaN
7,"Aircraft observed a uas while SW bound at 3,50...",AIRPORT DIRECTLY OVER,KUD,KUD,NaN,NaN,NaN,NaN,NaN
8,Aircraft observed a uas 100 feet above while W...,AIRPORT DIRECTLY OVER,HOK,HOK,NaN,NaN,NaN,NaN,NaN
9,Aircraft observed a black uas while SW bound a...,AIRPORT DIRECTLY OVER,RBV,RBV,NaN,NaN,NaN,NaN,NaN


In [25]:
air12_expdf = pd.merge(air12_exp, output72, on='REMARKS', how='left')
air12_expdf

,REMARKS,DATASET_x,UASLOCATION_x,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,index,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a uas off the left side whil...,AIRPORT DIRECTLY OVER,VIC,VIC,NaN,NaN,NaN,NaN,NaN,904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VIC,AIRPORT DIRECTLY OVER,NaN
1,Aircraft observed a white uas while W bound at...,AIRPORT DIRECTLY OVER,CRI,CRI,NaN,NaN,NaN,NaN,NaN,939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRI,AIRPORT DIRECTLY OVER,NaN
2,Aircraft observed a black/red uas off the righ...,AIRPORT DIRECTLY OVER,JAK,JAK,NaN,NaN,NaN,NaN,NaN,963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JAK,AIRPORT DIRECTLY OVER,NaN
3,Aircraft observed a uas off the right side whi...,AIRPORT DIRECTLY OVER,PXR,PXR,NaN,NaN,NaN,NaN,NaN,977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PXR,AIRPORT DIRECTLY OVER,NaN
4,"Aircraft observed a uas while S bound at 2,500...",AIRPORT DIRECTLY OVER,VAL,VAL,NaN,NaN,NaN,NaN,NaN,984,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VAL,AIRPORT DIRECTLY OVER,NaN
5,Aircraft observed a uas off the right side whi...,AIRPORT DIRECTLY OVER,CGT,CGT,NaN,NaN,NaN,NaN,NaN,1044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CGT,AIRPORT DIRECTLY OVER,NaN
6,Aircraft observed a black uas from the front s...,AIRPORT DIRECTLY OVER,CQY,CQY,NaN,NaN,NaN,NaN,NaN,1064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CQY,AIRPORT DIRECTLY OVER,NaN
7,"Aircraft observed a uas while SW bound at 3,50...",AIRPORT DIRECTLY OVER,KUD,KUD,NaN,NaN,NaN,NaN,NaN,1137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KUD,AIRPORT DIRECTLY OVER,NaN
8,Aircraft observed a uas 100 feet above while W...,AIRPORT DIRECTLY OVER,HOK,HOK,NaN,NaN,NaN,NaN,NaN,1158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOK,AIRPORT DIRECTLY OVER,NaN
9,Aircraft observed a black uas while SW bound a...,AIRPORT DIRECTLY OVER,RBV,RBV,NaN,NaN,NaN,NaN,NaN,1256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RBV,AIRPORT DIRECTLY OVER,NaN


In [26]:
# Export to .csv 
air12_expdf.to_csv('air12OF72_EXCEPTIONS.csv', index = False)

In [27]:
def link_airportsCleaned_ident(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='inner')

In [28]:
air36_link = link_airportsCleaned_ident(output72_airportDO)
air36_link

,REMARKS,DATASET,UASLOCATION,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed a black uas 4-5 feet in diam...,AIRPORT DIRECTLY OVER,DFW,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
1,"Aircraft observed a uas while WNW bound at 4,5...",AIRPORT DIRECTLY OVER,GSH,GSH,61E7F3D5-EF71-4A07-A204-30717CE5F692,Goshen Muni,41.526133,-85.794270,KGSH
2,"Aircraft reported a black, quad copter uas wit...",AIRPORT DIRECTLY OVER,CRG,CRG,0AAAC347-E1B1-4C82-8973-27920D299060,Jacksonville Exec At Craig,30.336333,-81.514444,KCRG
3,Aircraft reported a uas hovering over JFK whil...,AIRPORT DIRECTLY OVER,JFK,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK
4,Aircraft reported a uas or balloon while ESE a...,AIRPORT DIRECTLY OVER,RHV,RHV,B8849DA0-DA5F-4C6B-96ED-12E9C31F970C,Reid-Hillview of Santa Clara County,37.332861,-121.819806,KRHV
5,Aircraft reported a uas at the same altitude w...,AIRPORT DIRECTLY OVER,ABQ,ABQ,FC50BF53-167B-49DE-AF44-7D15FFC9E959,Albuquerque Intl Sunport,35.038932,-106.608262,KABQ
6,"Aircraft observed a black quad-copter uas 1,00...",AIRPORT DIRECTLY OVER,SFO,SFO,AF8BC34D-2D11-4CDB-9E87-33A0D430C055,San Francisco Intl,37.618806,-122.375417,KSFO
7,Aircraft observed a small uas off the front si...,AIRPORT DIRECTLY OVER,SCX,SCX,97B4CC8C-CC38-4D79-86F3-639102E2ACC2,Scott Muni,36.455694,-84.585750,KSCX
8,Aircraft observed a blue uas off the left side...,AIRPORT DIRECTLY OVER,VRB,VRB,F83408DF-2239-4254-9294-49C814E218A7,Vero Beach Rgnl,27.655556,-80.417944,KVRB
9,Aircraft observed a uas at the 12 o'clock posi...,AIRPORT DIRECTLY OVER,ATL,ATL,90091CE7-1372-4182-BA14-366919A3F134,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL


In [29]:
air36_link['UAS_Latitude'] = air36_link['LATITUDE']
air36_link['UAS_Longitude'] = air36_link['LONGITUDE']
air36_link

,REMARKS,DATASET,UASLOCATION,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Aircraft observed a black uas 4-5 feet in diam...,AIRPORT DIRECTLY OVER,DFW,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,32.897233,-97.037695
1,"Aircraft observed a uas while WNW bound at 4,5...",AIRPORT DIRECTLY OVER,GSH,GSH,61E7F3D5-EF71-4A07-A204-30717CE5F692,Goshen Muni,41.526133,-85.794270,KGSH,41.526133,-85.794270
2,"Aircraft reported a black, quad copter uas wit...",AIRPORT DIRECTLY OVER,CRG,CRG,0AAAC347-E1B1-4C82-8973-27920D299060,Jacksonville Exec At Craig,30.336333,-81.514444,KCRG,30.336333,-81.514444
3,Aircraft reported a uas hovering over JFK whil...,AIRPORT DIRECTLY OVER,JFK,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK,40.639928,-73.778693
4,Aircraft reported a uas or balloon while ESE a...,AIRPORT DIRECTLY OVER,RHV,RHV,B8849DA0-DA5F-4C6B-96ED-12E9C31F970C,Reid-Hillview of Santa Clara County,37.332861,-121.819806,KRHV,37.332861,-121.819806
5,Aircraft reported a uas at the same altitude w...,AIRPORT DIRECTLY OVER,ABQ,ABQ,FC50BF53-167B-49DE-AF44-7D15FFC9E959,Albuquerque Intl Sunport,35.038932,-106.608262,KABQ,35.038932,-106.608262
6,"Aircraft observed a black quad-copter uas 1,00...",AIRPORT DIRECTLY OVER,SFO,SFO,AF8BC34D-2D11-4CDB-9E87-33A0D430C055,San Francisco Intl,37.618806,-122.375417,KSFO,37.618806,-122.375417
7,Aircraft observed a small uas off the front si...,AIRPORT DIRECTLY OVER,SCX,SCX,97B4CC8C-CC38-4D79-86F3-639102E2ACC2,Scott Muni,36.455694,-84.585750,KSCX,36.455694,-84.585750
8,Aircraft observed a blue uas off the left side...,AIRPORT DIRECTLY OVER,VRB,VRB,F83408DF-2239-4254-9294-49C814E218A7,Vero Beach Rgnl,27.655556,-80.417944,KVRB,27.655556,-80.417944
9,Aircraft observed a uas at the 12 o'clock posi...,AIRPORT DIRECTLY OVER,ATL,ATL,90091CE7-1372-4182-BA14-366919A3F134,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL,33.636700,-84.427864


In [30]:
air36_linkdf = pd.merge(air36_link, output72, on='REMARKS', how='left')
air36_linkdf

,REMARKS,DATASET_x,UASLOCATION_x,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a black uas 4-5 feet in diam...,AIRPORT DIRECTLY OVER,DFW,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,32.897233,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DFW,AIRPORT DIRECTLY OVER,NaN
1,"Aircraft observed a uas while WNW bound at 4,5...",AIRPORT DIRECTLY OVER,GSH,GSH,61E7F3D5-EF71-4A07-A204-30717CE5F692,Goshen Muni,41.526133,-85.794270,KGSH,41.526133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GSH,AIRPORT DIRECTLY OVER,NaN
2,"Aircraft reported a black, quad copter uas wit...",AIRPORT DIRECTLY OVER,CRG,CRG,0AAAC347-E1B1-4C82-8973-27920D299060,Jacksonville Exec At Craig,30.336333,-81.514444,KCRG,30.336333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRG,AIRPORT DIRECTLY OVER,NaN
3,Aircraft reported a uas hovering over JFK whil...,AIRPORT DIRECTLY OVER,JFK,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK,40.639928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFK,AIRPORT DIRECTLY OVER,NaN
4,Aircraft reported a uas or balloon while ESE a...,AIRPORT DIRECTLY OVER,RHV,RHV,B8849DA0-DA5F-4C6B-96ED-12E9C31F970C,Reid-Hillview of Santa Clara County,37.332861,-121.819806,KRHV,37.332861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RHV,AIRPORT DIRECTLY OVER,NaN
5,Aircraft reported a uas at the same altitude w...,AIRPORT DIRECTLY OVER,ABQ,ABQ,FC50BF53-167B-49DE-AF44-7D15FFC9E959,Albuquerque Intl Sunport,35.038932,-106.608262,KABQ,35.038932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABQ,AIRPORT DIRECTLY OVER,NaN
6,"Aircraft observed a black quad-copter uas 1,00...",AIRPORT DIRECTLY OVER,SFO,SFO,AF8BC34D-2D11-4CDB-9E87-33A0D430C055,San Francisco Intl,37.618806,-122.375417,KSFO,37.618806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SFO,AIRPORT DIRECTLY OVER,NaN
7,Aircraft observed a small uas off the front si...,AIRPORT DIRECTLY OVER,SCX,SCX,97B4CC8C-CC38-4D79-86F3-639102E2ACC2,Scott Muni,36.455694,-84.585750,KSCX,36.455694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCX,AIRPORT DIRECTLY OVER,NaN
8,Aircraft observed a blue uas off the left side...,AIRPORT DIRECTLY OVER,VRB,VRB,F83408DF-2239-4254-9294-49C814E218A7,Vero Beach Rgnl,27.655556,-80.417944,KVRB,27.655556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRB,AIRPORT DIRECTLY OVER,NaN
9,Aircraft observed a uas at the 12 o'clock posi...,AIRPORT DIRECTLY OVER,ATL,ATL,90091CE7-1372-4182-BA14-366919A3F134,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL,33.636700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,AIRPORT DIRECTLY OVER,NaN


In [31]:
# Export to .csv 
air36_linkdf.to_csv('air36OF72_uasLatLong.csv', index = False)

# NAVAID DIRECTLY OVER

`These records will use the NAVAID System Lat/Long as the UAS Lat/Long, as the UAS was described as being 'directly over' the NAVAID System in question`

In [32]:
output72_navDO = output72_df[output72_df['DATASET'] == 'NAVAID DIRECTLY OVER'].drop(columns = ['cityname']).reset_index(drop=True)
output72_navDO

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a blue uas 200 feet below th...,NAVAID DIRECTLY OVER,CRG
1,Aircraft observed a white uas 5-10 feet at sam...,NAVAID DIRECTLY OVER,CAK
2,Aircraft observed a uas 30-50 feet off the lef...,NAVAID DIRECTLY OVER,ACO


In [33]:
output72_navDO['IDENT'] = output72_navDO['UASLOCATION']
output72_navDO

,REMARKS,DATASET,UASLOCATION,IDENT
0,Aircraft observed a blue uas 200 feet below th...,NAVAID DIRECTLY OVER,CRG,CRG
1,Aircraft observed a white uas 5-10 feet at sam...,NAVAID DIRECTLY OVER,CAK,CAK
2,Aircraft observed a uas 30-50 feet off the lef...,NAVAID DIRECTLY OVER,ACO,ACO


In [34]:
def nav_link (dataframe):
    '''
    This functions merges the dataframe input by the user with the cleaned NAVAID_cleaned.csv dataset
    '''
    
    # Read in Cleaned NAVAID Systems dataset
    nav_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')
    
    output_df = pd.merge(dataframe, nav_clean, on = 'IDENT', how = 'left').reset_index(drop=True)
    
    return output_df

In [35]:
nav3_link = nav_link(output72_navDO)
nav3_link

,REMARKS,DATASET,UASLOCATION,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,Aircraft observed a blue uas 200 feet below th...,NAVAID DIRECTLY OVER,CRG,CRG,-81.509930,30.338885,CRAIG,JACKSONVILLE
1,Aircraft observed a white uas 5-10 feet at sam...,NAVAID DIRECTLY OVER,CAK,CAK,NaN,NaN,NaN,NaN
2,Aircraft observed a uas 30-50 feet off the lef...,NAVAID DIRECTLY OVER,ACO,ACO,-81.201529,41.107911,AKRON,AKRON


In [36]:
nav1_exp = nav3_link.iloc[[1]]
nav1_exp

,REMARKS,DATASET,UASLOCATION,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
1,Aircraft observed a white uas 5-10 feet at sam...,NAVAID DIRECTLY OVER,CAK,CAK,NaN,NaN,NaN,NaN


In [37]:
nav1_expdf = pd.merge(nav1_exp, output72, on='REMARKS', how='left')
nav1_expdf

,REMARKS,DATASET_x,UASLOCATION_x,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,index,UAS_Location_Runways,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a white uas 5-10 feet at sam...,NAVAID DIRECTLY OVER,CAK,CAK,NaN,NaN,NaN,NaN,1048,UNKN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAK,NAVAID DIRECTLY OVER,NaN


In [38]:
# Export to .csv 
nav1_expdf.to_csv('nav01OF72_EXCEPTIONS.csv', index = False)

In [39]:
nav2 = nav3_link.iloc[[0, 2]]
nav2

,REMARKS,DATASET,UASLOCATION,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,Aircraft observed a blue uas 200 feet below th...,NAVAID DIRECTLY OVER,CRG,CRG,-81.509930,30.338885,CRAIG,JACKSONVILLE
2,Aircraft observed a uas 30-50 feet off the lef...,NAVAID DIRECTLY OVER,ACO,ACO,-81.201529,41.107911,AKRON,AKRON


In [40]:
nav2['UAS_Latitude'] = nav2['LATITUDE']
nav2['UAS_Longitude'] = nav2['LONGITUDE']
nav2

<ipython-input-40-7b8ed89348cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nav2['UAS_Latitude'] = nav2['LATITUDE']
<ipython-input-40-7b8ed89348cd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nav2['UAS_Longitude'] = nav2['LONGITUDE']


,REMARKS,DATASET,UASLOCATION,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude
0,Aircraft observed a blue uas 200 feet below th...,NAVAID DIRECTLY OVER,CRG,CRG,-81.509930,30.338885,CRAIG,JACKSONVILLE,30.338885,-81.509930
2,Aircraft observed a uas 30-50 feet off the lef...,NAVAID DIRECTLY OVER,ACO,ACO,-81.201529,41.107911,AKRON,AKRON,41.107911,-81.201529


In [41]:
nav2df = pd.merge(nav2, output72, on='REMARKS', how='left')
nav2df

,REMARKS,DATASET_x,UASLOCATION_x,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a blue uas 200 feet below th...,NAVAID DIRECTLY OVER,CRG,CRG,-81.509930,30.338885,CRAIG,JACKSONVILLE,30.338885,-81.509930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRG,NAVAID DIRECTLY OVER,NaN
1,Aircraft observed a uas 30-50 feet off the lef...,NAVAID DIRECTLY OVER,ACO,ACO,-81.201529,41.107911,AKRON,AKRON,41.107911,-81.201529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACO,NAVAID DIRECTLY OVER,NaN


In [42]:
# Export to .csv 
nav2df.to_csv('nav02OF72_uasLatLong.csv', index = False)

# NAVAID

In [43]:
output72_nav = output72_df[output72_df['DATASET'] == 'NAVAID'].drop(columns = ['cityname']).reset_index(drop=True)
output72_nav

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a uas at 200 feet above whil...,NAVAID,9 NM E of VNY
1,Aircraft observed a uas 100 feet below while W...,NAVAID,5 NM S of SJU
2,Aircraft reported a red and white quad copter ...,NAVAID,2.5 NM NE of EMT
3,"Aircraft observed a uas at 3,000 while SW boun...",NAVAID,10 NM S of MCI
4,Aircraft observed a black uas while S bound at...,NAVAID,20 NM ENE of DHP
5,Aircraft reported a white uas at the 3 o'clock...,NAVAID,5 NM N of FFU


In [44]:
nav6 = uas_loc_parts(output72_nav)
#nav6

In [45]:
nav6_link = nav_link(nav6)
nav6_link

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,Aircraft observed a uas at 200 feet above whil...,NAVAID,9 NM E of VNY,9,16.668,E,90,VNY,-118.491683,34.223467,VAN NUYS,VAN NUYS
1,Aircraft observed a uas 100 feet below while W...,NAVAID,5 NM S of SJU,5,9.260,S,180,SJU,NaN,NaN,NaN,NaN
2,Aircraft reported a red and white quad copter ...,NAVAID,2.5 NM NE of EMT,2.5,4.630,NE,45,EMT,NaN,NaN,NaN,NaN
3,"Aircraft observed a uas at 3,000 while SW boun...",NAVAID,10 NM S of MCI,10,18.520,S,180,MCI,-94.737071,39.285290,KANSAS CITY,KANSAS CITY
4,Aircraft observed a black uas while S bound at...,NAVAID,20 NM ENE of DHP,20,37.040,ENE,68,DHP,-80.349038,25.799966,DOLPHIN,MIAMI
5,Aircraft reported a white uas at the 3 o'clock...,NAVAID,5 NM N of FFU,5,9.260,N,0,FFU,-111.940543,40.274901,FAIRFIELD,PROVO


In [46]:
nav2_exp = nav6_link.iloc[[1,2]]
nav2_exp

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
1,Aircraft observed a uas 100 feet below while W...,NAVAID,5 NM S of SJU,5,9.26,S,180,SJU,NaN,NaN,NaN,NaN
2,Aircraft reported a red and white quad copter ...,NAVAID,2.5 NM NE of EMT,2.5,4.63,NE,45,EMT,NaN,NaN,NaN,NaN


In [47]:
nav2_expdf = pd.merge(nav2_exp, output72, on='REMARKS', how='left')
nav2_expdf

,REMARKS,DATASET_x,UASLOCATION_x,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a uas 100 feet below while W...,NAVAID,5 NM S of SJU,5,9.26,S,180,SJU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM S of SJU,NAVAID,NaN
1,Aircraft reported a red and white quad copter ...,NAVAID,2.5 NM NE of EMT,2.5,4.63,NE,45,EMT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5 NM NE of EMT,NAVAID,NaN


In [48]:
# Export to .csv 
nav2_expdf.to_csv('nav02OF72_EXCEPTIONS.csv', index = False)

In [49]:
nav4 = nav6_link.dropna().reset_index(drop=True)
#nav4

In [50]:
nav4_uas = uas_lat_long(nav4)
nav4_uas

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude
0,Aircraft observed a uas at 200 feet above whil...,NAVAID,9 NM E of VNY,9,16.668,E,90,VNY,-118.491683,34.223467,VAN NUYS,VAN NUYS,34.223334,-118.310789
1,"Aircraft observed a uas at 3,000 while SW boun...",NAVAID,10 NM S of MCI,10,18.520,S,180,MCI,-94.737071,39.285290,KANSAS CITY,KANSAS CITY,39.118473,-94.737071
2,Aircraft observed a black uas while S bound at...,NAVAID,20 NM ENE of DHP,20,37.040,ENE,68,DHP,-80.349038,25.799966,DOLPHIN,MIAMI,25.924808,-80.006231
3,Aircraft reported a white uas at the 3 o'clock...,NAVAID,5 NM N of FFU,5,9.260,N,0,FFU,-111.940543,40.274901,FAIRFIELD,PROVO,40.358294,-111.940543


In [51]:
nav4_uasdf = pd.merge(nav4_uas, output72, on='REMARKS', how='left')
nav4_uasdf

,REMARKS,DATASET_x,UASLOCATION_x,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a uas at 200 feet above whil...,NAVAID,9 NM E of VNY,9,16.668,E,90,VNY,-118.491683,34.223467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9 NM E of VNY,NAVAID,NaN
1,"Aircraft observed a uas at 3,000 while SW boun...",NAVAID,10 NM S of MCI,10,18.520,S,180,MCI,-94.737071,39.285290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM S of MCI,NAVAID,NaN
2,Aircraft observed a black uas while S bound at...,NAVAID,20 NM ENE of DHP,20,37.040,ENE,68,DHP,-80.349038,25.799966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20 NM ENE of DHP,NAVAID,NaN
3,Aircraft reported a white uas at the 3 o'clock...,NAVAID,5 NM N of FFU,5,9.260,N,0,FFU,-111.940543,40.274901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM N of FFU,NAVAID,NaN


In [52]:
# Export to .csv 
nav4_uasdf.to_csv('nav04OF72_uasLatLong.csv', index = False)

# FIX RADIAL

In [53]:
output72_fr = output72_df[output72_df['DATASET'] == 'FIX RADIAL'].drop(columns = ['cityname']).reset_index(drop=True)
output72_fr

,REMARKS,DATASET,UASLOCATION
0,Aircraft reported a uas off the left side whil...,FIX RADIAL,003 NM 170 of LGA


In [54]:
fr1 = uas_loc_parts(output72_fr)
fr1

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,Aircraft reported a uas off the left side whil...,FIX RADIAL,003 NM 170 of LGA,003,5.556,170,170,LGA


In [55]:
fr1_link = link_airportsCleaned_ident(fr1)
fr1_link

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft reported a uas off the left side whil...,FIX RADIAL,003 NM 170 of LGA,003,5.556,170,170,LGA,E866CAA3-781C-4162-8C31-D16CC9E3E92E,Laguardia,40.77725,-73.872611,KLGA


In [56]:
fr1_uas = uas_lat_long(fr1_link)
fr1_uas

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Aircraft reported a uas off the left side whil...,FIX RADIAL,003 NM 170 of LGA,003,5.556,170,170,LGA,E866CAA3-781C-4162-8C31-D16CC9E3E92E,Laguardia,40.77725,-73.872611,KLGA,40.727978,-73.861191


In [57]:
fr1_uasdf = pd.merge(fr1_uas, output72, on='REMARKS', how='left')
fr1_uasdf

,REMARKS,DATASET_x,UASLOCATION_x,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft reported a uas off the left side whil...,FIX RADIAL,003 NM 170 of LGA,003,5.556,170,170,LGA,E866CAA3-781C-4162-8C31-D16CC9E3E92E,Laguardia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,003 NM 170 of LGA,FIX RADIAL,NaN


In [58]:
# Export to .csv 
fr1_uasdf.to_csv('fr01OF72_uasLatLong.csv', index = False)

# RUNWAY

In [59]:
output72_rwy = output72_df[output72_df['DATASET'] == 'RUNWAY'].drop(columns = ['cityname']).reset_index(drop=True)
output72_rwy

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a black uas while on a 2 NM ...,RUNWAY,2 NM final for JFK
1,Aircraft observed a uas at same altitude while...,RUNWAY,DWH
2,"Aircraft observed a uas at 12 o clock, 100 fee...",RUNWAY,4 NM final to IAD
3,"Aircraft observed a 2 uas.s, grey and white, o...",RUNWAY,2 NM final to BWI


In [67]:
rwy4_exp = pd.merge(output72_rwy, output72, on='REMARKS', how='left')
rwy4_exp

,REMARKS,DATASET_x,UASLOCATION_x,index,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a black uas while on a 2 NM ...,RUNWAY,2 NM final for JFK,533,UNKN,5/31/2020,18:30,2020-05-31T18:30Z,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 31R,2 NM final for JFK,RUNWAY,NaN
1,Aircraft observed a uas at same altitude while...,RUNWAY,DWH,922,UNKN,12/29/2020,18:23,2020-12-29T18:23Z,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 17R,DWH,RUNWAY,NaN
2,"Aircraft observed a uas at 12 o clock, 100 fee...",RUNWAY,4 NM final to IAD,1376,UNKN,6/21/2021,13:50,2021-06-21T13:50Z,NCRCC,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 19L,4 NM final to IAD,RUNWAY,NaN
3,"Aircraft observed a 2 uas.s, grey and white, o...",RUNWAY,2 NM final to BWI,416,UNKN,2/16/2020,15:27,2020-02-16T15:27Z,NCRCC,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,RWY 33L,2 NM final to BWI,RUNWAY,NaN


In [68]:
# Export to .csv 
rwy4_exp.to_csv('rwy04OF72_EXCEPTIONS.csv', index = False)

# HELIPORT

In [60]:
output72_hp = output72_df[output72_df['DATASET'] == 'HELIPORT'].drop(columns = ['cityname']).reset_index(drop=True)
output72_hp

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a uas off 6 o'clock position...,HELIPORT,1 NM E of 5MI5


In [61]:
hp1 = uas_loc_parts(output72_hp)
hp1

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,Aircraft observed a uas off 6 o'clock position...,HELIPORT,1 NM E of 5MI5,1,1.852,E,90,5MI5


In [62]:
hp1_link = link_airportsCleaned_ident(hp1)
hp1_link

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed a uas off 6 o'clock position...,HELIPORT,1 NM E of 5MI5,1,1.852,E,90,5MI5,44269766-B09C-4CE9-A5FE-04374D05F8C4,Borgess Medical Center,42.308446,-85.560572,NaN


In [63]:
hp1_uas = uas_lat_long(hp1_link)
hp1_uas

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Aircraft observed a uas off 6 o'clock position...,HELIPORT,1 NM E of 5MI5,1,1.852,E,90,5MI5,44269766-B09C-4CE9-A5FE-04374D05F8C4,Borgess Medical Center,42.308446,-85.560572,NaN,42.308443,-85.53811


In [64]:
hp1_uasdf = pd.merge(hp1_uas, output72, on='REMARKS', how='left')
hp1_uasdf

,REMARKS,DATASET_x,UASLOCATION_x,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET_y,cityname
0,Aircraft observed a uas off 6 o'clock position...,HELIPORT,1 NM E of 5MI5,1,1.852,E,90,5MI5,44269766-B09C-4CE9-A5FE-04374D05F8C4,Borgess Medical Center,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 NM E of 5MI5,HELIPORT,NaN


In [65]:
# Export to .csv 
hp1_uasdf.to_csv('heliport01OF72_uasLatLong.csv', index = False)